<a href="https://colab.research.google.com/github/anarlavrenov/n1/blob/master/n1_training.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount("/content/drive")

Mounted at /content/drive


In [ ]:
import torch

device = "cuda" if torch.cuda.is_available() else "cpu"

In [ ]:
import sys
def init_packages() -> None:

  functions_path = "/PATH_TO_YOUR_PROJECT"
  sys.path.append(functions_path)

init_packages()

In [ ]:
!pip install datasets --quiet

from datasets import load_dataset
import pandas as pd
from typing import Tuple, List

def create_dataset(n_train_samples: int, n_valid_samples: int) -> Tuple[List, List]:

  train_dataset = load_dataset("d0p3/ukr-pravda-news-summary", split="train")

  train_df = pd.DataFrame(train_dataset)[:n_train_samples]
  valid_df = pd.DataFrame(train_dataset)[n_train_samples: n_train_samples + n_valid_samples]

  return train_df, valid_df

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 536.7/536.7 kB 10.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 11.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 13.1 MB/s eta 0:00:00


In [ ]:
import re

def preprocess_text(row: str) -> str:

  row = re.sub(r'https?:\/\/\S+|www\.[a-zA-Z0-9\-\.]+\.[a-zA-Z]+', '', row)
  row = re.sub(r'\s+', ' ', row).strip()

  return row

In [ ]:
from utils import PositionalEncoding
import math

class Encoder(torch.nn.Module):
  def __init__(self, num_layers: int, d_model: int, nhead: int,
               dff: int, ntokens: int, dropout: float = 0.5):
    super(Encoder, self).__init__()

    self.embedding = torch.nn.Embedding(num_embeddings=ntokens,
                                        embedding_dim=d_model,
                                        padding_idx=0)

    self.pos_encoding = PositionalEncoding(d_model=d_model,
                                           dropout=dropout)

    encoder_layer = torch.nn.TransformerEncoderLayer(d_model=d_model,
                                                           nhead=nhead,
                                                           dim_feedforward=dff,
                                                           dropout=dropout,
                                                           norm_first=True)

    self.encoder = torch.nn.TransformerEncoder(encoder_layer=encoder_layer,
                                               num_layers=num_layers)


    self.d_model = d_model

    self.linear_glu = torch.nn.Linear(in_features=d_model,
                    out_features=d_model * 2)

  def forward(self, src: torch.Tensor, mask: torch.Tensor = None) -> torch.Tensor:
    # src -> seq_len, batch_size, d_model
    src = self.embedding(src) * math.sqrt(self.d_model)
    src = self.pos_encoding(src)

    src = torch.nn.functional.glu(self.linear_glu(src), dim=-1) # Застосування GLU

    if mask is None:
      mask = torch.nn.Transformer.generate_square_subsequent_mask(sz=len(src)).to(device)

    encoder_output = self.encoder(src, mask)

    return encoder_output # -> Tensor shape: seq_len, batch_size, ntokens


class Decoder(torch.nn.Module):
  def __init__(self, num_layers: int, d_model: int, nhead: int,
               dff: int, ntokens: int, dropout: float = 0.5):
    super(Decoder, self).__init__()

    self.embedding = torch.nn.Embedding(num_embeddings=ntokens,
                                        embedding_dim=d_model,
                                        padding_idx=0)

    self.pos_encoding = PositionalEncoding(d_model=d_model,
                                           dropout=dropout)

    decoder_layer = torch.nn.TransformerDecoderLayer(d_model=d_model,
                                                      nhead=nhead,
                                                      dim_feedforward=dff,
                                                      dropout=dropout,
                                                      norm_first=True)

    self.decoder = torch.nn.TransformerDecoder(decoder_layer=decoder_layer,
                                               num_layers=num_layers)


    self.fc = torch.nn.Linear(in_features=d_model,
                              out_features=ntokens)

    self.d_model = d_model

    self.linear_glu = torch.nn.Linear(in_features=d_model,
                    out_features=d_model * 2)

  def forward(self, tgt: torch.Tensor, memory: torch.Tensor,
              tgt_mask: torch.Tensor = None, memory_mask: torch.Tensor = None):

    tgt = self.embedding(tgt) * math.sqrt(self.d_model)
    tgt = self.pos_encoding(tgt)

    tgt = torch.nn.functional.glu(self.linear_glu(tgt), dim=-1) # Застосування GLU

    if tgt_mask is None:
      tgt_mask = torch.nn.Transformer.generate_square_subsequent_mask(len(tgt)).to(device)

    if memory_mask is None:
      memory_mask = torch.zeros((tgt.size(1), memory.size(0))).to(device)

    decoder_output = self.decoder(tgt, memory,
                                  tgt_mask=tgt_mask, memory_key_padding_mask=memory_mask)


    output = self.fc(decoder_output)

    return output


class Transformer(torch.nn.Module):
  def __init__(self, num_layers_encoder: int, num_layers_decoder: int, d_model: int, nhead: int,
               dff: int, ntokens: int, dropout: float = 0.5):
    super(Transformer, self).__init__()

    self.encoder = Encoder(num_layers_encoder, d_model, nhead, dff, ntokens)
    self.decoder = Decoder(num_layers_decoder, d_model, nhead, dff, ntokens)


  def forward(self, src: torch.Tensor, tgt: torch.Tensor):

    memory = self.encoder(src)
    decoder_output = self.decoder(tgt, memory)

    return decoder_output

In [ ]:
train_df, valid_df = create_dataset(n_train_samples=50000,
                                    n_valid_samples=5000)

In [ ]:
# Лімітування довжин текстів через квантиль 80 відсотків для запобігання вибросам
import numpy as np

maxlen_text = int(np.quantile([len(x.split()) for x in train_df["text"]], q=0.8))
maxlen_title = int(np.quantile([len(x.split()) for x in train_df["summary"]], q=0.8))

train_df = train_df[train_df["text"].str.split().str.len() < maxlen_text]
train_df = train_df[train_df["summary"].str.split().str.len() < maxlen_title]

valid_df = valid_df[valid_df["text"].str.split().str.len() < maxlen_text]
valid_df = valid_df[valid_df["summary"].str.split().str.len() < maxlen_title]

In [ ]:
train_df["text"] = [preprocess_text(x) for x in train_df["text"]]
train_df["summary"] = [preprocess_text(x) for x in train_df["summary"]]


valid_df["text"] = [preprocess_text(x) for x in valid_df["text"]]
valid_df["summary"] = [preprocess_text(x) for x in valid_df["summary"]]

In [ ]:
train_df.shape, maxlen_text, maxlen_title

((32986, 2), 237, 33)

In [ ]:
# Формування функцій токенизації текстів

# !python -m spacy download uk_core_news_trf

import spacy
import torchtext
from torchtext.data.utils import get_tokenizer
from torchtext.vocab import build_vocab_from_iterator
from typing import Callable


def tokenize(input_data: List[str]) -> torch.Tuple[Callable[[str], List[str]], torchtext.vocab.Vocab]:
  spacy.prefer_gpu()
  nlp = spacy.load("uk_core_news_trf")

  def tokenizer(text: str) -> List[str]:
    return [tok.text for tok in nlp.tokenizer(text)]

  data_iter = iter(input_data)
  vocab = build_vocab_from_iterator(map(tokenizer, data_iter), specials=["<unk>"])
  vocab.set_default_index(vocab["<unk>"])

  return tokenizer, vocab


tokenizer, vocab = tokenize(train_df["text"] + train_df["summary"])

In [ ]:
# Формування датасету PyTorch

class DataWrapper(torch.utils.data.Dataset):
  def __init__(self, text: List[str], title: List[str]):
    super(DataWrapper, self).__init__()

    start_token = [len(vocab)]
    end_token = [len(vocab) + 1]

    self.text = text
    self.title = title

    self.text_ = [vocab(tokenizer(word)) for word in self.text]
    self.title_ = [vocab(tokenizer(word)) for word in self.title]

    self.text_ = np.asarray([self.pad_sequences(seq, maxlen_text,
                                                start_token, end_token) for seq in self.text_])
    self.title_ = np.asarray([self.pad_sequences(seq, maxlen_title,
                                                 start_token, end_token) for seq in self.title_])

  def __len__(self):

    return len(self.text_)


  def __getitem__(self, index: int):
    return self.text_[index], self.title_[index]


  def pad_sequences(self, seq, max_len: int, start_token, end_token):
    if max_len > len(seq):
      padding = [0] * (max_len - len(seq))

      return start_token + seq + end_token + padding

    else:
      return start_token + seq[:max_len] + end_token

In [ ]:
train_dataset = DataWrapper(train_df["text"],
                            train_df["summary"])

valid_dataset = DataWrapper(valid_df["text"],
                            valid_df["summary"])

In [ ]:
# Формування даталоадеру PyTorh
train_loader = torch.utils.data.DataLoader(train_dataset,
                                           batch_size=32,
                                           shuffle=True,
                                           num_workers=2,
                                           drop_last=True)

valid_loader = torch.utils.data.DataLoader(valid_dataset,
                                           batch_size=16,
                                           shuffle=False,
                                           num_workers=2,
                                           drop_last=True)

In [ ]:
# Ініціалізація трансформеру
num_layers_encoder = 6
num_layers_decoder = 6
d_model = 512
nhead = 8
dff = 1024
ntokens = len(vocab) + 2
dropout = 0.5

model = Transformer(num_layers_encoder, num_layers_decoder,
                    d_model, nhead, dff, ntokens, dropout=dropout)

for param in model.parameters():
  if param.dim() > 1:
    torch.nn.init.xavier_uniform_(param)

model = model.to(device)

/usr/local/lib/python3.10/dist-packages/torch/nn/modules/transformer.py:282: UserWarning: enable_nested_tensor is True, but self.use_nested_tensor is False because encoder_layer.norm_first was True
  warnings.warn(f"enable_nested_tensor is True, but self.use_nested_tensor is False because {why_not_sparsity_fast_path}")


In [ ]:
src = next(iter(train_loader))[0].long().to(device)
tgt = next(iter(train_loader))[1].long().to(device)

In [ ]:
enc_res = model.encoder(src.permute(1, 0))
dec_res = model.decoder(tgt.permute(1, 0), enc_res)

print(f" Вихідний розмір прогноза трансформеру: {dec_res.shape}, Початковий розмір таргету: {tgt.permute(1, 0).shape} \n"
      f" Такий розмір повинен мати pred: {dec_res.view(-1, ntokens).shape} "
      f"і таргет: {tgt.permute(1, 0).reshape(-1).shape} для функциї CrossEntropy")

(torch.Size([239, 32, 512]), torch.Size([35, 32]))

In [ ]:
lr = 0.1

criterion = torch.nn.CrossEntropyLoss(ignore_index=0)
optimizer = torch.optim.AdamW(model.parameters())
scheduler = torch.optim.lr_scheduler.StepLR(optimizer, 1.0, gamma=0.9)

In [ ]:
# Функція інференсу після навчання моделі

def summarize(string: str, model: torch.nn.Module,
              tokenizer: Callable[[str], List[str]], vocab: torchtext.vocab.Vocab,
              repetition_penalty: float = 1.2) -> torch.Tensor:

  model.eval()

  start_token = [len(vocab)]
  end_token = [len(vocab) + 1]

  string = torch.IntTensor([vocab(tokenizer(word))[0] for word in string.split()]).unsqueeze(0).to(device)
  output = torch.IntTensor(start_token).unsqueeze(0).to(device)

  with torch.no_grad():

    for i in range(maxlen_title):

      prediction = model(string.permute(1, 0), output.permute(1, 0))

      prediction = prediction[-1:, :, :]

      if i > 1:
        # repetition penalty
        for token_id in set(output.squeeze().tolist()):
          prediction[0, 0, token_id] /= repetition_penalty

      predicted_id = torch.argmax(prediction, dim=-1)

      if predicted_id[0] == end_token[0]:
        return output.squeeze(0)

      output = torch.cat([output, predicted_id.permute(1, 0)], dim=-1)

    return output.squeeze(0)

In [ ]:
# Функції навчання моделі на трейні та валідації
from tqdm import tqdm

def train(loader: torch.Tensor) -> float:

  model.train()

  total_loss = 0

  for batch in tqdm(loader):

    optimizer.zero_grad()

    src, tgt = batch[0].to(device), batch[1].to(device)

    tgt_inp = tgt[:, :-1].permute(1, 0)
    tgt_real = tgt[:, 1:].permute(1, 0)

    outputs = model(src.permute(1, 0), tgt_inp)
    loss = criterion(outputs.view(-1, ntokens), tgt_real.reshape(-1))

    total_loss += loss.item()

    loss.backward()
    torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=0.7)
    optimizer.step()

  return total_loss / len(loader)


def eval_(loader: torch.Tensor) -> float:

  model.eval()

  total_loss = 0

  with torch.no_grad():

    for batch in tqdm(loader):

      src, tgt = batch[0].to(device), batch[1].to(device)

      tgt_inp = tgt[:, :-1].permute(1, 0)
      tgt_real = tgt[:, 1:].permute(1, 0)

      outputs = model(src.permute(1, 0), tgt_inp)
      loss = criterion(outputs.view(-1, ntokens), tgt_real.reshape(-1))

      total_loss += loss.item()

  res = summarize(valid_df["text"].iloc[15], model=model,
                  tokenizer=tokenizer, vocab=vocab)
  # Відпринтовування поточного результату інференса моделі на даній епосі навчання
  print(" ".join([vocab.get_itos()[word] for word in res[1:]]))

  return total_loss / len(loader)

In [ ]:
# Запуск циклу навчання трансформерної моделі

epochs = 5

for epoch in range(epochs):
  loss = train(train_loader)
  valid_loss = eval_(valid_loader)
  print(f"epoch: {epoch + 1} | loss: {loss:.3f} | valid_loss: {valid_loss:.3f}")

  scheduler.step()

100%|██████████| 204/204 [00:05<00:00, 39.64it/s]


Російські окупанти обстріляли Херсон , внаслідок чого двоє чоловіків отримали поранення .
epoch: 1 | loss: 6.595 | valid_loss: 5.255


100%|██████████| 204/204 [00:05<00:00, 39.96it/s]


" За добу російські окупанти в тимчасово окупованому Криму , де вони не було знайдено ще один з яких були конфісковані у зв'язку із загрозою для України . "
epoch: 2 | loss: 4.738 | valid_loss: 4.723


100%|██████████| 204/204 [00:05<00:00, 39.14it/s]


Російські окупанти в тимчасово окупованому Луганську , щоб звинуватити в цьому не було втрачено понад 10 хвилин у війні проти України .
epoch: 3 | loss: 3.809 | valid_loss: 4.496


100%|██████████| 204/204 [00:05<00:00, 39.75it/s]


Російські окупанти захопили в окупованому Донецьку , де 12 березня біля окупованого Криму та Луганську переповнені пораненими російськими окупантами .
epoch: 4 | loss: 3.109 | valid_loss: 4.494


100%|██████████| 204/204 [00:05<00:00, 39.76it/s]


Російські окупанти захопили в окупованому місті Могоча обшуки у центрі міста Новоайдар на Харківщині .
epoch: 5 | loss: 2.577 | valid_loss: 4.538


In [ ]:
# Перевірка інференсу
res = summarize(valid_df["text"].iloc[7], model=model, tokenizer=tokenizer, vocab=vocab)

" ".join([vocab.get_itos()[word] for word in res[1:]])

'Держави - члени НАТО оголосили про початок навчань українських льотчиків на винищувачах F-16 , які будуть передані в Румунії .'

In [ ]:
valid_df["summary"].iloc[7]

'Держави, які входять до так званої "коаліції винищувачів", розглядають Румунію як можливе місце для навчання українських льотчиків керуванню винищувачами F-16.'

In [ ]:
# Зберігання результатів

import dill

torch.save(model, "/YOUR_PROJECT_PATH/model.pth")

torch.save(optimizer.state_dict(), "/YOUR_PROJECT_PATH/optimizer_state_dict.pth")

with open("/YOUR_PROJECT_PATH/vocab.pkl", "wb") as f:
  dill.dump(vocab, f)